In [0]:
!pip install -U -q PyDrive

In [19]:
from google.colab import files
uploaded = files.upload()

Saving amazon_jobs.csv to amazon_jobs.csv


In [0]:
filename = 'amazon_jobs.csv'

In [21]:
import pandas as pd

df = pd.read_csv(filename)
print(df.shape)
df.head()

(3493, 7)


,Unnamed: 0,Title,location,Posting_date,DESCRIPTION,BASIC QUALIFICATIONS,PREFERRED QUALIFICATIONS
0,0,Software Development Manager,"US, WA, Seattle","March 1, 2018",You are an experienced hands-on manager with a...,· Proven track record of hiring and managing h...,· Experience building extremely high volume an...
1,1,Software Development Engineer,"IN, KA, Bangalore","March 1, 2018",Amazon is driven by being “the world’s most cu...,· Bachelor’s Degree in Computer Science or rel...,· Experience building complex software systems...
2,2,Software Development Engineer,"IN, KA, Bangalore","March 1, 2018",Amazon is driven by being “the world’s most cu...,· Bachelor’s Degree in Computer Science or rel...,· Experience building complex software systems...
3,3,Software Development Engineer - Amazon Lex,"US, WA, Seattle","March 1, 2018",Have you ever wondered what it takes to build ...,· Bachelors Degree in Computer Science or a re...,· Masters or PhD in Computer Science· Experien...
4,4,Software Development Engineer - Amazon Lex,"US, WA, Seattle","March 1, 2018",Have you ever wondered what it takes to build ...,· Bachelors Degree in Computer Science or a re...,· Masters or PhD in Computer Science· Experien...


In [22]:
engg_jobs_df  = df[df['Title'].str.contains('Software') | df['Title'].str.contains('Developer')]
engg_jobs_df['Job Description'] = engg_jobs_df['DESCRIPTION'] + '\n' + engg_jobs_df['BASIC QUALIFICATIONS'] + '\n' + engg_jobs_df['PREFERRED QUALIFICATIONS']
jobs_df = pd.DataFrame(engg_jobs_df, columns=['Job Description'])
data = jobs_df.dropna()
print(data.shape)
data

(2998, 1)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Job Description
0,You are an experienced hands-on manager with a...
1,Amazon is driven by being “the world’s most cu...
2,Amazon is driven by being “the world’s most cu...
3,Have you ever wondered what it takes to build ...
4,Have you ever wondered what it takes to build ...
5,Have you ever wondered what it takes to build ...
7,The Amazon Devices team designs and engineers ...
8,The Amazon Devices team designs and engineers ...
9,The Amazon Devices team designs and engineers ...
10,Interested in Machine Learning? How about Amaz...


In [25]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

folder_list = drive.ListFile({'q': "'root' in parents and trashed=false and mimeType='application/vnd.google-apps.folder'"}).GetList()
for folder in folder_list:
  if folder['title'] == 'jobs-corpus':
    req_folder_id = folder['id']
    break


file_list = drive.ListFile({'q': "'%s' in parents and trashed=false" % req_folder_id}).GetList()
count = len(file_list) + 1
print("Existing file count:", count-1)

for i, row in data.iterrows():
  file_data = row[0]
  file_name = '%d.txt' % count
  print("writing %s to GDrive..." % file_name)
  export = drive.CreateFile({'title': file_name, 'parents': [{'kind': 'drive#fileLink', 'id': req_folder_id}]})
  export.SetContentString(file_data)
  export.Upload()
  count += 1

Existing file count: 29
writing 30.txt to GDrive...
writing 31.txt to GDrive...
writing 32.txt to GDrive...
writing 33.txt to GDrive...
writing 34.txt to GDrive...
writing 35.txt to GDrive...
writing 36.txt to GDrive...
writing 37.txt to GDrive...
writing 38.txt to GDrive...
writing 39.txt to GDrive...
writing 40.txt to GDrive...
writing 41.txt to GDrive...
writing 42.txt to GDrive...
writing 43.txt to GDrive...
writing 44.txt to GDrive...
writing 45.txt to GDrive...
writing 46.txt to GDrive...
writing 47.txt to GDrive...
writing 48.txt to GDrive...
writing 49.txt to GDrive...
writing 50.txt to GDrive...
writing 51.txt to GDrive...
writing 52.txt to GDrive...
writing 53.txt to GDrive...
writing 54.txt to GDrive...
writing 55.txt to GDrive...
writing 56.txt to GDrive...
writing 57.txt to GDrive...
writing 58.txt to GDrive...
writing 59.txt to GDrive...
writing 60.txt to GDrive...
writing 61.txt to GDrive...
writing 62.txt to GDrive...
writing 63.txt to GDrive...
writing 64.txt to GDrive

writing 113.txt to GDrive...
writing 114.txt to GDrive...
writing 115.txt to GDrive...
writing 116.txt to GDrive...
writing 117.txt to GDrive...
writing 118.txt to GDrive...
writing 119.txt to GDrive...
writing 120.txt to GDrive...
writing 121.txt to GDrive...
writing 122.txt to GDrive...
writing 123.txt to GDrive...
writing 124.txt to GDrive...
writing 125.txt to GDrive...
writing 126.txt to GDrive...
writing 127.txt to GDrive...
writing 128.txt to GDrive...
writing 129.txt to GDrive...
writing 130.txt to GDrive...
writing 131.txt to GDrive...
writing 132.txt to GDrive...
writing 133.txt to GDrive...
writing 134.txt to GDrive...
writing 135.txt to GDrive...
writing 136.txt to GDrive...
writing 137.txt to GDrive...
writing 138.txt to GDrive...
writing 139.txt to GDrive...
writing 140.txt to GDrive...
writing 141.txt to GDrive...
writing 142.txt to GDrive...
writing 143.txt to GDrive...
writing 144.txt to GDrive...
writing 145.txt to GDrive...
writing 146.txt to GDrive...
writing 147.tx

writing 195.txt to GDrive...
writing 196.txt to GDrive...
writing 197.txt to GDrive...
writing 198.txt to GDrive...
writing 199.txt to GDrive...
writing 200.txt to GDrive...
writing 201.txt to GDrive...
writing 202.txt to GDrive...
writing 203.txt to GDrive...
writing 204.txt to GDrive...
writing 205.txt to GDrive...
writing 206.txt to GDrive...
writing 207.txt to GDrive...
writing 208.txt to GDrive...
writing 209.txt to GDrive...
writing 210.txt to GDrive...
writing 211.txt to GDrive...
writing 212.txt to GDrive...
writing 213.txt to GDrive...
writing 214.txt to GDrive...
writing 215.txt to GDrive...
writing 216.txt to GDrive...
writing 217.txt to GDrive...
writing 218.txt to GDrive...
writing 219.txt to GDrive...
writing 220.txt to GDrive...
writing 221.txt to GDrive...
writing 222.txt to GDrive...
writing 223.txt to GDrive...
writing 224.txt to GDrive...
writing 225.txt to GDrive...
writing 226.txt to GDrive...
writing 227.txt to GDrive...
writing 228.txt to GDrive...
writing 229.tx

writing 277.txt to GDrive...
writing 278.txt to GDrive...
writing 279.txt to GDrive...
writing 280.txt to GDrive...
writing 281.txt to GDrive...
writing 282.txt to GDrive...
writing 283.txt to GDrive...
writing 284.txt to GDrive...
writing 285.txt to GDrive...
writing 286.txt to GDrive...
writing 287.txt to GDrive...
writing 288.txt to GDrive...
writing 289.txt to GDrive...
writing 290.txt to GDrive...
writing 291.txt to GDrive...
writing 292.txt to GDrive...
writing 293.txt to GDrive...
writing 294.txt to GDrive...
writing 295.txt to GDrive...
writing 296.txt to GDrive...
writing 297.txt to GDrive...
writing 298.txt to GDrive...
writing 299.txt to GDrive...
writing 300.txt to GDrive...
writing 301.txt to GDrive...
writing 302.txt to GDrive...
writing 303.txt to GDrive...
writing 304.txt to GDrive...
writing 305.txt to GDrive...
writing 306.txt to GDrive...
writing 307.txt to GDrive...
writing 308.txt to GDrive...
writing 309.txt to GDrive...
writing 310.txt to GDrive...
writing 311.tx

writing 359.txt to GDrive...
writing 360.txt to GDrive...
writing 361.txt to GDrive...
writing 362.txt to GDrive...
writing 363.txt to GDrive...
writing 364.txt to GDrive...
writing 365.txt to GDrive...
writing 366.txt to GDrive...
writing 367.txt to GDrive...
writing 368.txt to GDrive...
writing 369.txt to GDrive...
writing 370.txt to GDrive...
writing 371.txt to GDrive...
writing 372.txt to GDrive...
writing 373.txt to GDrive...
writing 374.txt to GDrive...
writing 375.txt to GDrive...
writing 376.txt to GDrive...
writing 377.txt to GDrive...
writing 378.txt to GDrive...
writing 379.txt to GDrive...
writing 380.txt to GDrive...
writing 381.txt to GDrive...
writing 382.txt to GDrive...
writing 383.txt to GDrive...
writing 384.txt to GDrive...
writing 385.txt to GDrive...
writing 386.txt to GDrive...
writing 387.txt to GDrive...
writing 388.txt to GDrive...
writing 389.txt to GDrive...
writing 390.txt to GDrive...
writing 391.txt to GDrive...
writing 392.txt to GDrive...
writing 393.tx

...

writing 2636.txt to GDrive...
writing 2637.txt to GDrive...
writing 2638.txt to GDrive...
writing 2639.txt to GDrive...
writing 2640.txt to GDrive...
writing 2641.txt to GDrive...
writing 2642.txt to GDrive...
writing 2643.txt to GDrive...
writing 2644.txt to GDrive...
writing 2645.txt to GDrive...
writing 2646.txt to GDrive...
writing 2647.txt to GDrive...
writing 2648.txt to GDrive...
writing 2649.txt to GDrive...
writing 2650.txt to GDrive...
writing 2651.txt to GDrive...
writing 2652.txt to GDrive...
writing 2653.txt to GDrive...
writing 2654.txt to GDrive...
writing 2655.txt to GDrive...
writing 2656.txt to GDrive...
writing 2657.txt to GDrive...
writing 2658.txt to GDrive...
writing 2659.txt to GDrive...
writing 2660.txt to GDrive...
writing 2661.txt to GDrive...
writing 2662.txt to GDrive...
writing 2663.txt to GDrive...
writing 2664.txt to GDrive...
writing 2665.txt to GDrive...
writing 2666.txt to GDrive...
writing 2667.txt to GDrive...
writing 2668.txt to GDrive...
writing 26

writing 2717.txt to GDrive...
writing 2718.txt to GDrive...
writing 2719.txt to GDrive...
writing 2720.txt to GDrive...
writing 2721.txt to GDrive...
writing 2722.txt to GDrive...
writing 2723.txt to GDrive...
writing 2724.txt to GDrive...
writing 2725.txt to GDrive...
writing 2726.txt to GDrive...
writing 2727.txt to GDrive...
writing 2728.txt to GDrive...
writing 2729.txt to GDrive...
writing 2730.txt to GDrive...
writing 2731.txt to GDrive...
writing 2732.txt to GDrive...
writing 2733.txt to GDrive...
writing 2734.txt to GDrive...
writing 2735.txt to GDrive...
writing 2736.txt to GDrive...
writing 2737.txt to GDrive...
writing 2738.txt to GDrive...
writing 2739.txt to GDrive...
writing 2740.txt to GDrive...
writing 2741.txt to GDrive...
writing 2742.txt to GDrive...
writing 2743.txt to GDrive...
writing 2744.txt to GDrive...
writing 2745.txt to GDrive...
writing 2746.txt to GDrive...
writing 2747.txt to GDrive...
writing 2748.txt to GDrive...
writing 2749.txt to GDrive...
writing 27

writing 2799.txt to GDrive...
writing 2800.txt to GDrive...
writing 2801.txt to GDrive...
writing 2802.txt to GDrive...
writing 2803.txt to GDrive...
writing 2804.txt to GDrive...
writing 2805.txt to GDrive...
writing 2806.txt to GDrive...
writing 2807.txt to GDrive...
writing 2808.txt to GDrive...
writing 2809.txt to GDrive...
writing 2810.txt to GDrive...
writing 2811.txt to GDrive...
writing 2812.txt to GDrive...
writing 2813.txt to GDrive...
writing 2814.txt to GDrive...
writing 2815.txt to GDrive...
writing 2816.txt to GDrive...
writing 2817.txt to GDrive...
writing 2818.txt to GDrive...
writing 2819.txt to GDrive...
writing 2820.txt to GDrive...
writing 2821.txt to GDrive...
writing 2822.txt to GDrive...
writing 2823.txt to GDrive...
writing 2824.txt to GDrive...
writing 2825.txt to GDrive...
writing 2826.txt to GDrive...
writing 2827.txt to GDrive...
writing 2828.txt to GDrive...
writing 2829.txt to GDrive...
writing 2830.txt to GDrive...
writing 2831.txt to GDrive...
writing 28

writing 2880.txt to GDrive...
writing 2881.txt to GDrive...
writing 2882.txt to GDrive...
writing 2883.txt to GDrive...
writing 2884.txt to GDrive...
writing 2885.txt to GDrive...
writing 2886.txt to GDrive...
writing 2887.txt to GDrive...
writing 2888.txt to GDrive...
writing 2889.txt to GDrive...
writing 2890.txt to GDrive...
writing 2891.txt to GDrive...
writing 2892.txt to GDrive...
writing 2893.txt to GDrive...
writing 2894.txt to GDrive...
writing 2895.txt to GDrive...
writing 2896.txt to GDrive...
writing 2897.txt to GDrive...
writing 2898.txt to GDrive...
writing 2899.txt to GDrive...
writing 2900.txt to GDrive...
writing 2901.txt to GDrive...
writing 2902.txt to GDrive...
writing 2903.txt to GDrive...
writing 2904.txt to GDrive...
writing 2905.txt to GDrive...
writing 2906.txt to GDrive...
writing 2907.txt to GDrive...
writing 2908.txt to GDrive...
writing 2909.txt to GDrive...
writing 2910.txt to GDrive...
writing 2911.txt to GDrive...
writing 2912.txt to GDrive...
writing 29

writing 2961.txt to GDrive...
writing 2962.txt to GDrive...
writing 2963.txt to GDrive...
writing 2964.txt to GDrive...
writing 2965.txt to GDrive...
writing 2966.txt to GDrive...
writing 2967.txt to GDrive...
writing 2968.txt to GDrive...
writing 2969.txt to GDrive...
writing 2970.txt to GDrive...
writing 2971.txt to GDrive...
writing 2972.txt to GDrive...
writing 2973.txt to GDrive...
writing 2974.txt to GDrive...
writing 2975.txt to GDrive...
writing 2976.txt to GDrive...
writing 2977.txt to GDrive...
writing 2978.txt to GDrive...
writing 2979.txt to GDrive...
writing 2980.txt to GDrive...
writing 2981.txt to GDrive...
writing 2982.txt to GDrive...
writing 2983.txt to GDrive...
writing 2984.txt to GDrive...
writing 2985.txt to GDrive...
writing 2986.txt to GDrive...
writing 2987.txt to GDrive...
writing 2988.txt to GDrive...
writing 2989.txt to GDrive...
writing 2990.txt to GDrive...
writing 2991.txt to GDrive...
writing 2992.txt to GDrive...
writing 2993.txt to GDrive...
writing 29